In [1]:
import astropy.io.fits as pyfits
import os
import numpy as np
from scipy.misc import imresize
import pandas as pd

In [40]:
label_file = "R_filtrid/BD_decomp_asi.txt"
galaxies_info = pd.read_table(label_file, sep='\t', lineterminator='\n',skiprows=1,header=None,index_col=False,
                              names=["cnr","n","kval","varb","dust","popul","rescale"],
                             usecols=["cnr","kval","varb","dust","popul","rescale"])
galaxies_info['LABEL'] =  np.array(galaxies_info['kval']>1.5).astype(int)
#galaxies_info['cnr'] = galaxies_info['cnr'].apply(str)
#galaxies_info = galaxies_info.sort_values("cnr")
galaxies_info.head(10)

,cnr,kval,varb,dust,popul,rescale,LABEL
0,105,2.0,0,T,T,F,1
1,106,1.0,1,F,F,F,0
2,107,2.0,0.2,F,T,F,1
3,108,1.5,0,F,T,F,0
4,113,2.0,0,F,T,F,1
5,114,1.5,0.2,F,T,F,0
6,115,2.0,0,F,F,T,1
7,116,1.7,0,T,T,T,1
8,117,1.8,0,F,T,F,1
9,118,2.0,0,T,T,F,1


In [45]:
labels = np.array(galaxies_info['LABEL'])
sum(labels)/len(labels)

0.18207282913165265

In [41]:
galaxies_fits = []
max_shape0 = 0
max_shape1 = 0
im_size = 128
path = "R_filtrid/"
for num in galaxies_info['cnr']:
    #file_names.append(file)
    fits_file = pyfits.getdata(path+str(num)+".fits")
    compressed_file = imresize(fits_file,int(im_size*100/max(fits_file.shape)))
    galaxies_fits.append(compressed_file)
    if compressed_file.shape[0]>max_shape0:
        max_shape0=compressed_file.shape[0]
    if compressed_file.shape[1]>max_shape1:
        max_shape1=compressed_file.shape[1]

In [42]:
max_shape0, max_shape1

(128, 128)

In [44]:
galaxies = []
for arr in galaxies_fits:
    padded_arr = np.pad(arr,((im_size-arr.shape[0], 0), (im_size-arr.shape[1], 0)), 'constant')
    galaxies.append(padded_arr[:, :, np.newaxis])

In [45]:
galaxies[0].shape

(128, 128, 1)

In [54]:
galaxies_array = np.array(galaxies)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    rotation_range=20,
    width_shift_range=0.5,
    height_shift_range=0.5,
    zoom_range = 0.5,
    fill_mode = 'constant',
    vertical_flip = True,
    horizontal_flip=True)

In [50]:
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, Flatten, Dense, AveragePooling2D, MaxPooling2D
from keras.optimizers import Adam

# We first need to define the sequence of dependencies (the computational graph)
x = Input(shape=(im_size, im_size, 1))
h = Conv2D(96, (8, 8))(x)
h = Activation('relu')(h)
h = MaxPooling2D(pool_size = (3, 3))(h)
h = Flatten()(h)
h = Dense(24)(h)
h = Activation('relu')(h)
h = Dense(2)(h)
p = Activation('softmax')(h)

# Now that we have defined how to find p from x, we can create a 
# model simply by saying what is input and what is output
model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 121, 121, 96)      6240      
_________________________________________________________________
activation_4 (Activation)    (None, 121, 121, 96)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 40, 40, 96)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 153600)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                3686424   
_________________________________________________________________
activation_5 (Activation)    (None, 24)                0         
__________

In [52]:
# old
model.fit(galaxies_array, labels,
          epochs=50,
          batch_size=25,
          validation_split=0.2)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 834 arrays: [array([[[0],
        [0],
        [0],
        ..., 
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ..., 
        [0],
        [0],
        [0]],

       [[0]...

In [ ]:
datagen.fit(galaxies_array)

In [ ]:
batch_size=32
class_weight = {0 : 1., 1: len(labels)/sum(labels)}
model.fit_generator(datagen.flow(galaxies_array, labels, batch_size = batch_size),
                    steps_per_epoch = len(x_train)/batch_size, epochs = 30,
                    validation_split = 0.2, class_weight = class_weight)